In [31]:
import pandas as pd
import numpy as np
from pandas import DataFrame as df
import math, sys, os
import random as rand
from itertools import combinations, permutations
import pickle as pkl
import threading
from sklearn.metrics import silhouette_score,davies_bouldin_score,calinski_harabasz_score
from sklearn.cluster import KMeans, Birch
from sklearn import metrics
import copy
from numpy import random
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import AgglomerativeClustering
import matplotlib.cm as cm
from multiprocessing import Process
from tabulate import tabulate
import warnings
import statistics
import warnings
window_size=5

In [32]:
class RCAC:
  def __init__(self,Dataset_name,split_size,num_clusters,data_drop_columns, compare_with_others = False, trials = 400,num_threads = 1000) -> None:
    ## Make changes only in this section. Preferably don't change path
    ## The code will automatically make the directories if they don't exist.

    self.save_location = '.temp/'
    self.trials = trials
    self.num_threads = num_threads
    self.labels_ = []
    self.best_so_far = 0
    self.Dataset_name = Dataset_name
    self.rule_kind = 'best'
    self.split_size = split_size
    self.num_clusters = num_clusters
    self.split_index = 0
    self.compare = compare_with_others
    self.data_drop_columns = data_drop_columns
    warnings.filterwarnings("ignore")


  def fit(self):
    rule_list_name = self.rule_kind+'_cycles_'+str(self.split_size)
    os.makedirs('./'+self.save_location+self.Dataset_name+'/Custers-'+str(self.num_clusters)+'/Final Clusters', exist_ok=True)
    os.makedirs('./config/'+self.Dataset_name+'/Custers-'+str(self.num_clusters), exist_ok=True)

    self.data=pd.read_csv("./data/"+self.Dataset_name+".csv")
    self.data=self.data.dropna()
    self.data=self.data.drop(self.data_drop_columns,axis=1)
    window_size = 5
    #encoder = BiNCE(self.data, length_of_each=2)
    #self.enc = encoder.encode()
    # if you want to apply freq based encoding,
    self.enc = freq_encoding(self.data)
    #self.enc = list(self.enc[self.enc.columns[0]])
    #print(self.enc)

    split_enc, num_of_splits = split_string(self.enc,self.split_size)
    rule_list = self.get_rule_list('./rules/'+rule_list_name+'.txt')
    rules_comb = list(combinations(rule_list, 2))


    self.cellular_automata_clustering(rule_list,num_of_splits,rules_comb , split_enc, window_size)


  def get_rule_list(self,path="./rule_list.txt"):
      # check if best rules exist
      best_path = './config/'+self.Dataset_name+'/Custers-'+str(self.num_clusters)+'/'+self.rule_kind+'_rules.txt'
      try:
        print("Found best configuration!")
        my_file = open(best_path, 'r')
        file_con = my_file.read()
        # replacing end splitting the text

        rule_list = file_con.split("\n")
        my_file.close()
        self.num_threads = 1
        self.trials = 2
        return rule_list
      except:
        print("No best rules yet. Running trials...")
      # opening the file in read mode
      my_file = open(path, "r")
      file_con = my_file.read()
      rule_list = file_con.split("\n")
      my_file.close()
      return rule_list
  def aggregate_scores(self):
    path = './'+self.save_location+self.Dataset_name+'/Custers-'+str(self.num_clusters)+'/'
    main_df = pd.DataFrame()
    for files in os.listdir(path):
        if '.DS_Store' in files or '.csv' not in files or 'final_scores' in files:
            continue
        df = pd.read_csv(path+files,index_col=False)
        main_df = main_df.append(df)

    main_df.columns = ["Index","Rule 1", "Rule 2", "Rule 3", "CA Silhoutte","Heir Silhoutte", "Kmeans Silhoutte", "Birch Silhoutte"]
    main_df = main_df.sort_values('CA Silhoutte', ascending=False)
    main_df.reset_index(inplace = True, drop=True)
    print(main_df)

    best_score = main_df['CA Silhoutte'].iloc[0]
    best_rules = [main_df['Rule 1'].iloc[0], main_df['Rule 2'].iloc[0]]
    main_df.to_csv('./'+self.save_location+'/'+self.Dataset_name+'/Custers-'+str(self.num_clusters)+'/'+self.rule_kind+'_final_scores.csv')
    return best_score, best_rules


#**********************Our Proposed Algorithm Starts Here***************************

  def cellular_automata_clustering(self,rule_list, split, rules_comb, encoding, window_size = 5):
    better_score_list = []
    best_CA_sill = -10000
    best_rule = []
    output_data = []
    columns = ["Rule 1", "Rule 2", "Rule 3", "CA Silhoutte","Heir Silhoutte", "Kmeans Silhoutte", "Birch Silhoutte"]



    rule_list_name = self.rule_kind+'_cycles_'+str(self.split_size)

    rule_list3=copy.deepcopy(rule_list)
    print("Rule3list",rule_list3)

    for rule_set in rules_comb:
      for rule3 in rule_list3:
        enc1 = copy.deepcopy(encoding)
#***********************************************Stage1***************************************************************
        for p in range(int(len(rule_set)/2)):
          rule = rg(int(rule_set[0]), window_size)
          fc = Stage1(rule,enc1,split)

#***********************************************Stage2***************************************************************
          rule=rg(int(rule_set[1]),window_size)

          R3=rg(int(rule3),window_size)
          stage2_output=[]
          stage2_output=Stage2(fc,enc1,rule,R3)
          stage2_dataset=stage2_output[0]
          stage2_cycles=stage2_output[1]

#******************************************************Stage3************************************************************************************
          enc_data_=Stage3(stage2_dataset,stage2_cycles,self.num_clusters)
          X=self.data.to_numpy()
          try :
            CA_sill_new=silhouette_score(X,enc_data_,metric="euclidean")
            print("davies:CA",davies_bouldin_score(X,enc_data_))
            print("calinski:CA",calinski_harabasz_score(X,enc_data_))
          except:
            CA_sill_new = 0



          #### ADD OTHER CLUSTERING METHODS FOR COMPARISON HERE ########

          ## Heirarchical
          clusterer = AgglomerativeClustering(n_clusters=self.num_clusters, affinity='euclidean', linkage='ward')
          cluster_labels = clusterer.fit_predict(X)
          try:
            Heir_sill_new = silhouette_score(X, cluster_labels)
            print("davies:Heirarchical",davies_bouldin_score(X,cluster_labels))
            print("calinski:Heirarchical",calinski_harabasz_score(X,cluster_labels))
          except:
            Heir_sill_new = 0


          ## Kmeans
          clust=KMeans(n_clusters=self.num_clusters,random_state=42)
          clust.fit(X)
          Km_labels=clust.labels_
          print(Km_labels)
          try:
            Kmeans_sill_new=metrics.silhouette_score(X, Km_labels, metric='euclidean')
            print("davies:Kmeans",davies_bouldin_score(X,Km_labels))
            print("calinski:Kmeans",calinski_harabasz_score(X,Km_labels))
          except:
            Kmeans_sill_new = 0



          ## Birch
          clust_model = Birch(n_clusters=self.num_clusters,branching_factor=1500,threshold=1.5)
          clust_model.fit(X)
          labels = clust_model.labels_
          try:
            Birch_new=metrics.silhouette_score(X, labels, metric='euclidean')
            print("davies:Birch",davies_bouldin_score(X,labels))
            print("calinski:Birch",calinski_harabasz_score(X,labels))
          except Exception as e:

            Birch_new = 0



          # Add rto output data
          output_data.append([rule_set[0],rule_set[1],rule3,CA_sill_new,Heir_sill_new, Kmeans_sill_new, Birch_new])
          out_df = pd.DataFrame(data=output_data, columns=columns)
          out_df.to_csv('./'+self.save_location+'/'+self.Dataset_name+'/Custers-'+str(self.num_clusters)+'/best_'+str(self.split_index)+'_tr_'+str(1)+'.csv')

          best_score, best_rules  = self.aggregate_scores()
          if self.best_so_far < best_score:
            self.best_so_far = best_score
            print("Best silhoutte score :", self.best_so_far)
            self.labels_ = enc_data_
            with open('./config/'+self.Dataset_name+'/Custers-'+str(self.num_clusters)+'/'+self.rule_kind+'_rules.txt', 'w') as file1:
              file1.write(str(best_rules[0])+"\n"+str(best_rules[1])+"\n"+str(rule3))

          if self.compare:
            my_data=[[self.num_clusters,self.best_so_far,Heir_sill_new, Kmeans_sill_new, Birch_new]]
            head=["Final no.of clusters"," Our silhoutte score","Heirarchical", "Kmeans", "Birch"]
            print(tabulate(my_data, headers=head, tablefmt="grid"))



          print("---------------------***-----------------******----------------------***--------------------")



In [33]:
def freq_encoding(data):
    encoder=['00','01','11','10']

    len(data)

    #Frequency Based Encoding
    details= dict()
    for j in data.columns:
      col=data[j]
      sortx = dict()
      a=list()
      for i in col:
        if i not in sortx:
          sortx[i] = 1
          a.append(i)
        else:
          sortx[i]+=1
      a.sort()
      p=0
      s=0
      i=0
      l=0
      while p<len(data):
        m=int(sortx[a[l]])
        if  s+m <=(math.ceil(len(data)/len(encoder))) :
          s+=m
          sortx[a[l]]=encoder[i]
        else:
          if l == 0:
            sortx[a[l]]=encoder[i]
            i+=1
          else:
            i+=1
            sortx[a[l]]=encoder[i]
          s=0
        l+=1
        p+=m
      details[j]=sortx

    #Replacing the data in table with encoding
    data1 = df(columns = data.columns)
    for i in range(len(data)):
      t=[]
      p=data.loc[i]
      for j in range(len(p)):
        s=details[data.columns[j]]
        t.append(s[p[j]])
      ar = pd.Series(t, index = data1.columns)
      data1 = data1.append(ar,ignore_index = True)
      #pd.concat([data1,ar])



    #Generating final encoded string
    enc=[]
    for i in range(len(data1)):
      p=data1.loc[i]
      t=""
      for j in p:
        t+=j
      enc.append(t)
    enc
    return enc

In [34]:
def split_string(encodings, div):
    r_ind=[]
    enc_length = len(encodings[0])
    # div = math.floor((enc_length)/split)+1
    for i in range(div, enc_length, div):
      r_ind.append(i)
    iclust=[]
    for i in range(len(encodings)):
      s=0
      for j in r_ind:
        iclust.append(encodings[i][s:j])
        s=j
      iclust.append(encodings[i][s:])
      encodings[i]=iclust
      iclust=[]
    return encodings, len(r_ind)+1

In [35]:
def rg(rule, winsize):
    brule=bin(rule).replace("0b","").zfill(2**winsize)
    brule=brule[::-1]
    return brule

In [36]:
def Stage1(rule,enc_stage1,split):
  fc = {}
  tr = []

  #***********************applying rule to each split********************

  for i in range(split):
    for j in range(len(enc_stage1)):
      tr.append(enc_stage1[j][i])
    fc[i]=apply_rule(tr,window_size,rule)
    tr = []
  return fc

In [37]:
def Stage2(cluster1,dataset1,R2,R3):

  #***************Sorting the cluster based on median**********************

  for i in range(len(cluster1)):
    icluster_Med_sort=Median_Cycles(cluster1[i])
    iclust=copy.deepcopy(icluster_Med_sort)
    s=findn(len(iclust))

    #*****************changing the datset elements with cluster number****************
    for j in range(len(dataset1)):
      plt=dataset1[j][i]
      l=0
      while(plt not in iclust[l]):
        l+=1
      dataset1[j][i]= cy_enc(s,l)



  #************************************merging all split into one****************
  for i in range(len(dataset1)):
    iclust=""
    for j in dataset1[i]:
      iclust+=j
    dataset1[i]=iclust
  enc_data=[]
  init_clusters=[]

  #If merged data length less than maximum possible cell size  apply rule 2
  if(len(dataset1[0])<=13):
    init_clusters=apply_rule(dataset1,window_size,R2)
    init_cluster_Med_sort=Median_Cycles(init_clusters)
    enc_data=copy.deepcopy(dataset1)
    for i in range(len(dataset1)):
      j=0
      while j <len(init_cluster_Med_sort):
        if dataset1[i] in init_cluster_Med_sort[j]:
          dataset1[i]=j
          break
        else:
          j+=1
  else:
    Stage2_output_reduced=Stage_part2(dataset1,R3)
    return Stage2_output_reduced
  Stage2_output=[]
  Stage2_output.append(dataset1)
  Stage2_output.append(init_cluster_Med_sort)
  return Stage2_output

In [38]:
def Stage_part2(dataset2,R3):
  print("Rule 3 Used")
  split_size=13
  split_enc2, num_of_splits2 = split_string(dataset2,split_size)
  Stage2_part2_output1=Stage1(R3,split_enc2,num_of_splits2)
  stage2_output2=[]
  stage2_output2=Stage2(Stage2_part2_output1,dataset2,R3,R3)
  return stage2_output2

In [39]:
def Stage3(ST3_in,cycle_3,num_clust):
  #Find decimal values of cluster for sorting
  if(len(cycle_3)<num_clust):
    return
  Dec_CL=[]
  for ele in cycle_3:
    L1=[]
    for e in ele:
      L1.append(binaryTodecimal(int(e)))
    Dec_CL.append(L1)
    Median_CL=[]
  for ele in Dec_CL:
    Median_CL.append((statistics.median(ele)))
  Unsorted_median=copy.deepcopy(Median_CL)
  Median_CL.sort()
  gap=[]
  for i in range(len(Median_CL)-1):
     gap.append(Median_CL[i+1]-Median_CL[i])
  unsortgap=copy.deepcopy(gap)
  gap.sort(reverse=True)
  gap_index=[]
  for i in range(num_clust-1):
    gap_index.append(unsortgap.index(gap[i]))
  gap_index.sort()
  flag=[]
  for i in range(len(gap_index)+1):
    if(i==0):
      for j in range(len(ST3_in)):
        if((ST3_in[j]<=gap_index[i]) and (j not in flag)):
          ST3_in[j]=i
          flag.append(j)
    elif(i>0 and i<(len(gap_index))):
      for j in range(len(ST3_in)):
        if((gap_index[i-1]< ST3_in[j] <= gap_index[i]) and (j not in flag)):
          ST3_in[j]=i
          flag.append(j)
    elif(i==(len(gap_index))):
      for j in range(len(ST3_in)):
        if((gap_index[i-1]<=ST3_in[j]) and (j not in flag)):
          ST3_in[j]=i
          flag.append(j)
  print(ST3_in)
  return(ST3_in)

In [40]:
def apply_rule(split,winsize,brule):
    final_array = []
    split_list=list(set(split))
    split_list.sort(reverse=True)
    split_len=len(split_list[0])
    current_array=[]
    while(split_list):
      curr_element=split_list[0]
      flag=0
      while(not flag):
        if current_array == []:
          current_array.append(curr_element)
          split_list.remove(curr_element)
        else:
          curr_element=nullbound(split_len,winsize,curr_element)
          t2=""
          for j in range(split_len):
            check=int(str(curr_element[j:j+winsize]),2)
            t2+=brule[check]
          curr_element=t2
          if curr_element not in current_array:
              if curr_element in split_list:
                split_list.remove(curr_element)
                current_array.append(curr_element)
          else:
              flag=1
      final_array.append(current_array)
      current_array=[]
    return final_array

In [41]:
def findn(d):
      i=1
      while 2**i<d:
        i+=1
      return i

In [42]:
def cy_enc(l,index):
      #return bin(index).replace("0b","").zfill(l)
      n=index
      gray=n ^ (n >> 1)
      return bin(gray).replace("0b","").zfill(l)

In [43]:
def Median_Cycles(FFCC):
  len1=len(FFCC[0][0])
  Dec_FFCC=[]
  for ele in FFCC:
    L1=[]
    for e in ele:
      L1.append(binaryTodecimal(int(e)))
    Dec_FFCC.append(L1)
    Median_FFCC=[]
  for ele in Dec_FFCC:
    Median_FFCC.append((statistics.median(ele)))
  for i in range(0, len(Median_FFCC)):
    for j in range(i+1, len(Median_FFCC)):
        if Median_FFCC[i] >= Median_FFCC[j]:
            Median_FFCC[i], Median_FFCC[j] = Median_FFCC[j],Median_FFCC[i]
            Dec_FFCC[i], Dec_FFCC[j] = Dec_FFCC[j],Dec_FFCC[i]
  bin_FFCC=[]
  for ele in Dec_FFCC:
    L1=[]
    for e in ele:
      L1.append(toBinary(e, len1))
    bin_FFCC.append(L1)
  return(bin_FFCC)

In [44]:
def nullbound(n,winsize,init_clusters_):
    if n%2!=0:
            if winsize%2!=0:
              init_clusters_ = "0"*(winsize//2) + init_clusters_ + "0"*(winsize//2)
            else:
              init_clusters_ = "0"*((winsize//2)+1) + init_clusters_ + "0"*(winsize//2)
    else:
            if winsize%2!=0:
              init_clusters_ = "0"*(winsize//2) + init_clusters_ + "0"*(winsize//2)
            else:
              init_clusters_ = "0"*((winsize//2)+1) + init_clusters_ + "0"*(winsize//2)
    return init_clusters_

In [45]:
def binaryTodecimal(n):
    decimal = 0
    power = 1
    while n>0:
        rem = n%10
        n = n//10
        decimal += rem*power
        power = power*2
    return decimal

In [46]:
def toBinary(n, len):
    binary = ''
    i = 1 << len - 1
    while i > 0:
        binary += '1' if (n & i) else '0'
        i = i // 2
    return binary

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [48]:
RCA = RCAC('school_district_breakdowns_12', 12, 2, data_drop_columns=['jurisdiction_name'], compare_with_others = True)
RCA.fit()

FileNotFoundError: [Errno 2] No such file or directory: './data/school_district_breakdowns_12.csv'

In [ ]:
RCA = RCAC('buddymove_holiday_10',10, 2, data_drop_columns=['User Id'], compare_with_others = True)
RCA.fit()

In [ ]:
RCA = RCAC('IPL - IPL_11', 11, 2, data_drop_columns=['PLAYER'], compare_with_others = True)
RCA.fit()

In [ ]:
RCA = RCAC('Iris_12', 12, 2, data_drop_columns=['Species'], compare_with_others = True)
RCA.fit()

In [ ]:
RCA = RCAC('school_district_breakdowns_12', 12, 2, data_drop_columns=['jurisdiction_name'], compare_with_others = True)
RCA.fit()

In [ ]:
#RCA = RCAC('buddymove_holidayiq', 13, 2, data_drop_columns=['User Id'], compare_with_others = True)
#RCA.fit()

In [ ]:
#RCA = RCAC('heart_failure_clinical_records_dataset', 13, 2, data_drop_columns=['DEATH_EVENT'], compare_with_others = True)
#RCA.fit()

In [ ]:
#RCA = RCAC('sobar-72CervicalCancerBehaviorRisk', 13, 2, data_drop_columns=['ca_cervix'], compare_with_others = True)
#RCA.fit()

In [ ]:
#RCA = RCAC('seeds_dataset - seeds_dataset', 13, 2, data_drop_columns=['class'], compare_with_others = True)
#RCA.fit()

In [ ]:
#RCA = RCAC('Wholesale customers data', 13, 2, data_drop_columns=['Channel'], compare_with_others = True)
#RCA.fit()

In [ ]:
#RCA = RCAC('StoneFlakes', 13, 2, data_drop_columns=['ID'], compare_with_others = True)
#RCA.fit()

In [ ]:
RCA = RCAC('Iris_8',8, 2, data_drop_columns=['Species'], compare_with_others = True)
RCA.fit()